In [1]:
import pandas as pd
# !pip install rake_nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from rake_nltk import Rake
rake = Rake()

In [2]:
ml_25 = pd.read_csv('10mil.csv')

In [9]:
ml_10 = pd.read_csv('movies.dat',sep='::',names=['movieId','title','genres'],encoding='latin-1')

D:\Python 3.7\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
data = ml_25[['title','director','cast','genres','overview']]
data.head(3)

,title,director,cast,genres,overview
0,Toy Story,John Lasseter,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,Animation|Comedy|Family,Woody the cowboy is young Andyâ€™s favorite to...
1,Jumanji,Joe Johnston,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Adventure|Fantasy|Family,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,Howard Deutch,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,Romance|Comedy,A family wedding reignites the ancient feud be...


In [5]:
ml_25['year'].max()

2008

In [4]:
data['director'] = data['director'].fillna(' ')
data['director'] = data['director'].astype(str)
data['cast'] = data['cast'].fillna(' ')
data['cast'] = data['cast'].astype(str)
data['key_notes'] = ''
for index,row in data.iterrows():
    plot = str(row['overview'])
    rake.extract_keywords_from_text(plot)
    keyword_score = rake.get_word_degrees()
    genre = ' '.join(str(row['genres']).split('|')).lower()
    director = ' '.join(row['director'].replace(' ','').split('|')).lower()
    cast = ' '.join(row['cast'].replace(' ','').split('|')).lower()
    keyword_score = ' '.join(list(keyword_score.keys()))
    row['key_notes'] = genre + ' ' + ' ' + director + ' ' + cast + ' ' + keyword_score
recommend = data[['title','key_notes']]
recommend['key_notes']

D:\Python 3.7\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Python 3.7\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Python 3.7\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

0        animation comedy family  johnlasseter tomhanks...
1        adventure fantasy family  joejohnston robinwil...
2        romance comedy  howarddeutch waltermatthau jac...
3        comedy drama romance  forestwhitaker whitneyho...
4        comedy  charlesshyer stevemartin dianekeaton m...
                               ...                        
10676    fantasy comedy family romance  adamshankman ad...
10677                                 none  none none none
10678    comedy drama  clarkgregg anjelicahuston kellym...
10679    drama romance  sammendes leonardodicaprio kate...
10680    comedy history  paulweiland rowanatkinson tony...
Name: key_notes, Length: 10681, dtype: object

In [5]:
cv = CountVectorizer()
count_mat = cv.fit_transform(recommend['key_notes'])
cosine_sim = cosine_similarity(count_mat,count_mat)
print(cosine_sim)

[[1.         0.02354408 0.04819627 ... 0.03256448 0.02951407 0.08623253]
 [0.02354408 1.         0.06978632 ... 0.         0.02849014 0.        ]
 [0.04819627 0.06978632 1.         ... 0.03217447 0.02916059 0.03407991]
 ...
 [0.03256448 0.         0.03217447 ... 1.         0.03940552 0.04605313]
 [0.02951407 0.02849014 0.02916059 ... 0.03940552 1.         0.        ]
 [0.08623253 0.         0.03407991 ... 0.04605313 0.         1.        ]]


In [19]:
indices = pd.Series(recommend['title'])
def recommend_movie(name):
    movie=[]
    idx_full = indices[indices == name].index
    idx = indices[indices == name].index[0]
    print(idx_full)
    sort_index = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    recommendation_5= sort_index.iloc[1:5]
    for i in recommendation_5.index:
        movie.append(indices[i])
    return movie

In [24]:
recommend_movie('Heat')

Int64Index([5], dtype='int64')


['Narrow Margin', 'Kiss Kiss Bang Bang', 'Thief', 'Manhunter']

In [ ]:
def rec():
    try:
        i = 1
        while(i > 0):
            name = input("Enter The Name of a Movie or Tv Show: ")
            if name.lower() == 'quit':
                break
            else:
                print(recommend_movie(name))

    except KeyboardInterrupt:
        print("The movie or Tv Show does not exist\n")
        rec()

    except IndexError:
        print("The movie or Tv Show does not exist\n")
        rec()
        

print("To exit Enter \"quit\" \n")
rec()

To exit Enter "quit" 

Enter The Name of a Movie or Tv Show: The Irishman
The movie or Tv Show does not exist

Enter The Name of a Movie or Tv Show: Toy Story
['Toy Story 2', "Doug's 1st Movie", 'Oliver & Company', 'American Pop']
Enter The Name of a Movie or Tv Show: Jumanji
['MirrorMask', 'Bedknobs and Broomsticks', "Lemony Snicket's A Series of Unfortunate Events", 'Hook']


In [15]:
ml_25[ml_25['title'].str.startswith('Harry',na=False)==True]

,movieId,title,Genre,year,cast,director,tagline,overview,runtime,genres,production_companies
3302,3388,Harry and the Hendersons,Children|Comedy,1987,John Lithgow|Melinda Dillon|Margaret Langrick|...,William Dear,"When You Can't Believe Your Eyes, Trust Your H...","Returning from a hunting trip in the forest, t...",110.0,Comedy|Family|Fantasy,Universal Pictures|Amblin Entertainment
4802,4896,Harry Potter and the Sorcerer's Stone,Adventure|Children|Fantasy,2001,None,None,None,None,None,None,None
5391,5487,Harry and Walter Go to New York,Comedy,1976,None,None,None,None,None,None,None
5718,5816,Harry Potter and the Chamber of Secrets,Adventure|Children|Fantasy,2002,Daniel Radcliffe|Rupert Grint|Emma Watson|Kenn...,Chris Columbus,Hogwarts is back in session.,"Ignoring threats to his life, Harry returns to...",161.0,Adventure|Fantasy|Family,1492 Pictures|Warner Bros.|Heyday Films|MIRACL...
7754,8368,Harry Potter and the Prisoner of Azkaban,Adventure|Children|Fantasy,2004,Daniel Radcliffe|Rupert Grint|Emma Watson|Gary...,Alfonso CuarÃ³n,Something wicked this way comes.,"Harry, Ron and Hermione return to Hogwarts for...",141.0,Adventure|Fantasy|Family,1492 Pictures|Warner Bros.|Heyday Films|P of A...
9335,40815,Harry Potter and the Goblet of Fire,Adventure|Fantasy|Thriller,2005,Daniel Radcliffe|Rupert Grint|Emma Watson|Ralp...,Mike Newell,Dark And Difficult Times Lie Ahead.,"Harry starts his fourth year at Hogwarts, comp...",157.0,Adventure|Fantasy|Family,Patalex IV Productions Limited|Warner Bros.|He...
9941,54001,Harry Potter and the Order of the Phoenix,Adventure|Drama|Fantasy|IMAX,2007,Daniel Radcliffe|Rupert Grint|Emma Watson|Mich...,David Yates,Evil Must Be Confronted.,Returning for his fifth year of study at Hogwa...,138.0,Adventure|Fantasy|Family|Mystery,Cool Music|Warner Bros.|Heyday Films|Harry Pot...
